In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk

In [3]:
from nltk.corpus import stopwords
from textblob import Word
from textblob import TextBlob
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
spotify=pd.read_csv("Dataset/reviews.csv",usecols=["Review","Rating"]) # Reading the dataset and using only the review and rating columns
data=spotify.copy()
data.head()

,Review,Rating
0,"Great music service, the audio is high quality...",5
1,Please ignore previous negative rating. This a...,5
2,"This pop-up ""Get the best Spotify experience o...",4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didn't ...,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61594 entries, 0 to 61593
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  61594 non-null  object
 1   Rating  61594 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 962.5+ KB


remove Punctuation

In [7]:
import re
data["Review"]=data["Review"].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))
data.head(5)

,Review,Rating
0,Great music service the audio is high quality ...,5
1,Please ignore previous negative rating This ap...,5
2,This pop-up Get the best Spotify experience on...,4
3,Really buggy and terrible to use as of recently,1
4,Dear Spotify why do I get songs that I didnt p...,1


lowercase

In [8]:
data["Review"]=data["Review"].apply(lambda x: " ".join(x.lower() for x in x.split()))
data.head(5)

,Review,Rating
0,great music service the audio is high quality ...,5
1,please ignore previous negative rating this ap...,5
2,this pop-up get the best spotify experience on...,4
3,really buggy and terrible to use as of recently,1
4,dear spotify why do i get songs that i didnt p...,1


whitespace removal

In [10]:
data["Review"]=data["Review"].apply(lambda x: " ".join(x.split()))
data.head(5)

,Review,Rating
0,great music service the audio is high quality ...,5
1,please ignore previous negative rating this ap...,5
2,this pop-up get the best spotify experience on...,4
3,really buggy and terrible to use as of recently,1
4,dear spotify why do i get songs that i didnt p...,1


remove numbers

In [ ]:
#remove numbers
data["Review"]=data["Review"].str.replace